In [75]:
import hopsworks
from hsfs.client.exceptions import RestAPIError

# Establish connection to Hopsworks
project = hopsworks.login(
    api_key_value="dYTVrTVvbj6Qw82i.YGKHdS9snQYFgOADJIvLdvZ2n2S5BxIAOtvPUEmAyd56bvaG6xhhGyNM3nYbexaP"
)

# Access the Feature Store
fs = project.get_feature_store()

# List of feature groups and their versions
feature_groups = [
    {"name": "city_weather", "version": 1},
    {"name": "drivers", "version": 1},
    {"name": "routes", "version": 1},
    {"name": "routes_weather", "version": 1},
    {"name": "traffic", "version": 1},  # Corrected typo
    {"name": "truck_schedule", "version": 1},
    {"name": "trucks", "version": 1},
]
# Initialize an empty dictionary to hold DataFrames for each feature group
feature_group_data = {}

# Loop over the list of feature groups and download them
for fg in feature_groups:
    try:
        # Retrieve the feature group by its name and version
        feature_group = fs.get_feature_group(fg['name'], version=fg['version'])
        df = feature_group.read()  # Read the feature group as a DataFrame
        
        # Save the DataFrame in the dictionary with the feature group's name as the key
        feature_group_data[fg['name']] = df

        print(f"Downloaded feature group: {fg['name']} (version {fg['version']})")
        print(df.head())  # Optionally display the first few rows of the DataFrame

    except RestAPIError as e:
        print(f"Error downloading feature group: {fg['name']} (version {fg['version']})")
        print(e)

# After the loop, you will have each feature group stored in the `feature_group_data` dictionary
# Access each DataFrame like this: feature_group_data['city_weather'], etc.



Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1044630
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.44s) 
Downloaded feature group: city_weather (version 1)
   index_column     city_id  temp  wind_speed    description  precip  \
0         26374  C-19236709    55          10          Sunny     0.0   
1         37883  C-e6dcda92    19          17          Sunny     0.0   
2         21614  C-419cd14c    57           9          Sunny     0.0   
3          9578  C-ef47bdcd    36           7  Partly cloudy     0.0   
4         29488  C-34ae77b0    79           6          Clear     0.0   

   humidity  visibility  pressure  chanceofrain  chanceoffog  chanceofsnow  \
0        62           6      1001             0            0             0   
1        74           6      1027             0  

In [76]:
print(feature_group_data.keys())


dict_keys(['city_weather', 'drivers', 'routes', 'routes_weather', 'traffic', 'truck_schedule', 'trucks'])


In [77]:
city_weather = feature_group_data['city_weather']
drivers = feature_group_data['drivers']
routes = feature_group_data['routes']
routes_weather = feature_group_data['routes_weather']
traffic = feature_group_data['traffic'] 
truck_schedule = feature_group_data['truck_schedule']
trucks = feature_group_data['trucks']


In [78]:
def assign_feature_group_data(feature_group_data, table_names):
    # Dynamically assign the DataFrame from feature_group_data to the variable names provided in table_names
    for table_name in table_names:
        globals()[table_name] = feature_group_data[table_name]
        print(f"Assigned {table_name} to variable.")
        
# List of table names to be assigned
table_names = ['city_weather', 'routes_weather', 'traffic', 'truck_schedule', 'trucks']

# Call the function to assign feature group data to variables
assign_feature_group_data(feature_group_data, table_names)

# Now you can access the variables dynamically assigned
print(truck_schedule.head())        

# 'drivers_table', 'routes_table'


Assigned city_weather to variable.
Assigned routes_weather to variable.
Assigned traffic to variable.
Assigned truck_schedule to variable.
Assigned trucks to variable.
   index_column  truck_id    route_id       departure_date  \
0          5832  46155960  R-c3d9f83b  2019-02-03 07:00:00   
1          1825  23024395  R-f782724c  2019-01-10 07:00:00   
2          9186  23608280  R-4d09a561  2019-01-17 07:00:00   
3          3063  28641704  R-a55547f0  2019-01-16 07:00:00   
4          9578  13426754  R-006548e0  2019-02-10 07:00:00   

               estimated_arrival  delay                event_time  
0  2019-02-03 08:31:48.000000000      1 2024-09-17 00:00:00+00:00  
1  2019-01-10 11:55:48.000000000      1 2024-09-17 00:00:00+00:00  
2  2019-01-19 14:35:24.000000000      0 2024-09-17 00:00:00+00:00  
3  2019-01-16 19:36:00.000000000      1 2024-09-17 00:00:00+00:00  
4  2019-02-11 23:01:12.000000000      1 2024-09-17 00:00:00+00:00  


In [79]:

dfs = {
    'city_weather': city_weather,
    'drivers': drivers,
    'routes': routes,
    'routes_weather': routes_weather,
    'traffic': traffic,
    'truck_schedule': truck_schedule,
    'trucks': trucks,
}


def drop_columns_from_tables(dfs, columns_to_drop):
    """
    Drops specified columns from all DataFrames in the provided dictionary.
    
    Parameters:
    - dfs (dict): A dictionary where the keys are DataFrame names and the values are pandas DataFrames.
    - columns_to_drop (list): A list of column names to be dropped from each DataFrame.
    
    Returns:
    - dict: A dictionary of DataFrames with the specified columns removed.
    """
    
    # Loop through each DataFrame in the dictionary
    for table_name, df in dfs.items():
        # Check if the columns exist in the DataFrame before dropping
        cols_to_drop = [col for col in columns_to_drop if col in df.columns]
        
        # If there are columns to drop, drop them
        if cols_to_drop:
            dfs[table_name] = df.drop(columns=cols_to_drop, inplace=True)
            print(f"Dropped columns {cols_to_drop} from {table_name}.")
        else:
            print(f"No matching columns to drop in {table_name}.")
    
    return dfs

# Example usage:
columns_to_drop = ['index_column', 'event_time']  # Example columns that might be present in multiple tables
feature_group_data_cleaned = drop_columns_from_tables(feature_group_data, columns_to_drop)




Dropped columns ['index_column', 'event_time'] from city_weather.
Dropped columns ['index_column', 'event_time'] from drivers.
Dropped columns ['index_column', 'event_time'] from routes.
Dropped columns ['index_column', 'event_time'] from routes_weather.
Dropped columns ['index_column', 'event_time'] from traffic.
Dropped columns ['index_column', 'event_time'] from truck_schedule.
Dropped columns ['index_column', 'event_time'] from trucks.


In [80]:
def drop_duplicates_from_dfs(dfs, columns_to_check):
    for df_name, df in dfs.items():
        if df_name in columns_to_check:
            subset_columns = columns_to_check[df_name]
            
            # Check if all columns in subset_columns exist in the DataFrame
            missing_columns = [col for col in subset_columns if col not in df.columns]
            if missing_columns:
                print(f"Warning: DataFrame {df_name} does not contain the columns: {missing_columns}")
                continue  # Skip this DataFrame and move to the next one
            
            before_dropping = df.shape[0]  # Count the number of rows before dropping duplicates
            df.drop_duplicates(subset=subset_columns, inplace=True)
            after_dropping = df.shape[0]  # Count the number of rows after dropping duplicates
            print(f"Dropped {before_dropping - after_dropping} duplicate rows from {df_name} based on columns: {subset_columns}")
        else:
            print(f"No duplicate check columns provided for {df_name}")

# Dictionary where the key is the DataFrame name, and the value is the subset of columns to check for duplicates
columns_to_check = {
    'city_weather': ['city_id', 'datetime'],
    'routes_weather': ['route_id', 'date'],
    'trucks': ['truck_id'],
    'drivers': ['driver_id'],
    'routes': ['route_id', 'destination_id', 'origin_id'],
    'truck_schedule': ['truck_id', 'route_id', 'departure_date']
}

# Call the function to drop duplicates from the DataFrames
drop_duplicates_from_dfs(dfs, columns_to_check)

# Check the result by printing the modified DataFrames
for df_name, df in dfs.items():
    print(f"\n{df_name} DataFrame after dropping duplicates:")
    print(df)


Dropped 707 duplicate rows from city_weather based on columns: ['city_id', 'datetime']


Dropped 0 duplicate rows from drivers based on columns: ['driver_id']
Dropped 0 duplicate rows from routes based on columns: ['route_id', 'destination_id', 'origin_id']
Dropped 0 duplicate rows from routes_weather based on columns: ['route_id', 'date']
No duplicate check columns provided for traffic
Dropped 0 duplicate rows from truck_schedule based on columns: ['truck_id', 'route_id', 'departure_date']
Dropped 0 duplicate rows from trucks based on columns: ['truck_id']

city_weather DataFrame after dropping duplicates:
          city_id  temp  wind_speed        description  precip  humidity  \
0      C-19236709    55          10              Sunny     0.0        62   
1      C-e6dcda92    19          17              Sunny     0.0        74   
2      C-419cd14c    57           9              Sunny     0.0        70   
3      C-ef47bdcd    36           7      Partly cloudy     0.0        53   
4      C-34ae77b0    79           6              Clear     0.0        45   
...           ...

In [81]:
city_weather.head(5)

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,chanceofrain,chanceoffog,chanceofsnow,chanceofthunder,datetime
0,C-19236709,55,10,Sunny,0.0,62,6,1001,0,0,0,0,2019-02-14 13:00:00+00:00
1,C-e6dcda92,19,17,Sunny,0.0,74,6,1027,0,0,0,0,2019-01-24 11:00:00+00:00
2,C-419cd14c,57,9,Sunny,0.0,70,6,1029,0,0,0,0,2019-01-09 06:00:00+00:00
3,C-ef47bdcd,36,7,Partly cloudy,0.0,53,6,1005,0,0,0,0,2019-01-06 07:00:00+00:00
4,C-34ae77b0,79,6,Clear,0.0,45,6,1016,0,0,0,0,2019-01-12 20:00:00+00:00


In [82]:
feature_group_data_cleaned['city_weather']

In [83]:
city_weather.tail()

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,chanceofrain,chanceoffog,chanceofsnow,chanceofthunder,datetime
38835,C-2aaf0e1a,23,3,Patchy light snow,0.0,61,6,1027,0,0,0,0,2019-01-13 23:00:00+00:00
38836,C-b6e04c88,30,6,Patchy light snow,0.0,87,6,1019,0,0,0,0,2019-01-26 22:00:00+00:00
38837,C-c7cacd1d,52,11,Sunny,0.0,79,6,1011,0,0,0,0,2019-02-04 16:00:00+00:00
38838,C-2c9e75ef,57,17,Partly cloudy,0.0,88,6,1009,0,0,0,0,2019-01-06 22:00:00+00:00
38839,C-594514f8,37,12,Overcast,0.0,93,6,1012,0,0,0,0,2019-02-04 20:00:00+00:00


In [84]:
#code to drop the unnecessary columns in the city_weather dataframe
# List of columns to drop
columns_to_drop = ['chanceofrain', 'chanceoffog', 'chanceofsnow', 'chanceofthunder']

# Drop the specified columns from the city_weather DataFrame
city_weather.drop(columns=columns_to_drop, inplace=True)
routes_weather.drop(columns=columns_to_drop, inplace=True)

# Display the updated DataFrame
city_weather.head()
routes_weather.head()



,route_id,date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,R-df8c8ec9,2019-02-10 00:00:00+00:00,70,6,Sunny,0.0,62,6,1015
1,R-38d1e82d,2019-01-20 06:00:00+00:00,19,4,Clear,0.0,88,6,1021
2,R-593b486f,2019-02-13 18:00:00+00:00,63,3,Clear,0.0,88,6,1022
3,R-89580831,2019-01-29 06:00:00+00:00,68,2,Sunny,0.0,44,6,1017
4,R-da652dd4,2019-02-07 18:00:00+00:00,70,10,Sunny,0.0,46,6,1018


In [85]:
city_weather

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,C-19236709,55,10,Sunny,0.0,62,6,1001,2019-02-14 13:00:00+00:00
1,C-e6dcda92,19,17,Sunny,0.0,74,6,1027,2019-01-24 11:00:00+00:00
2,C-419cd14c,57,9,Sunny,0.0,70,6,1029,2019-01-09 06:00:00+00:00
3,C-ef47bdcd,36,7,Partly cloudy,0.0,53,6,1005,2019-01-06 07:00:00+00:00
4,C-34ae77b0,79,6,Clear,0.0,45,6,1016,2019-01-12 20:00:00+00:00
...,...,...,...,...,...,...,...,...,...
38835,C-2aaf0e1a,23,3,Patchy light snow,0.0,61,6,1027,2019-01-13 23:00:00+00:00
38836,C-b6e04c88,30,6,Patchy light snow,0.0,87,6,1019,2019-01-26 22:00:00+00:00
38837,C-c7cacd1d,52,11,Sunny,0.0,79,6,1011,2019-02-04 16:00:00+00:00
38838,C-2c9e75ef,57,17,Partly cloudy,0.0,88,6,1009,2019-01-06 22:00:00+00:00


In [86]:
# Rename the 'date' column to 'custom_date'
city_weather.rename(columns={'datetime': 'custom_date'}, inplace=True)

# Display the updated DataFrame
city_weather.head()

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,custom_date
0,C-19236709,55,10,Sunny,0.0,62,6,1001,2019-02-14 13:00:00+00:00
1,C-e6dcda92,19,17,Sunny,0.0,74,6,1027,2019-01-24 11:00:00+00:00
2,C-419cd14c,57,9,Sunny,0.0,70,6,1029,2019-01-09 06:00:00+00:00
3,C-ef47bdcd,36,7,Partly cloudy,0.0,53,6,1005,2019-01-06 07:00:00+00:00
4,C-34ae77b0,79,6,Clear,0.0,45,6,1016,2019-01-12 20:00:00+00:00


In [87]:
city_weather

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,custom_date
0,C-19236709,55,10,Sunny,0.0,62,6,1001,2019-02-14 13:00:00+00:00
1,C-e6dcda92,19,17,Sunny,0.0,74,6,1027,2019-01-24 11:00:00+00:00
2,C-419cd14c,57,9,Sunny,0.0,70,6,1029,2019-01-09 06:00:00+00:00
3,C-ef47bdcd,36,7,Partly cloudy,0.0,53,6,1005,2019-01-06 07:00:00+00:00
4,C-34ae77b0,79,6,Clear,0.0,45,6,1016,2019-01-12 20:00:00+00:00
...,...,...,...,...,...,...,...,...,...
38835,C-2aaf0e1a,23,3,Patchy light snow,0.0,61,6,1027,2019-01-13 23:00:00+00:00
38836,C-b6e04c88,30,6,Patchy light snow,0.0,87,6,1019,2019-01-26 22:00:00+00:00
38837,C-c7cacd1d,52,11,Sunny,0.0,79,6,1011,2019-02-04 16:00:00+00:00
38838,C-2c9e75ef,57,17,Partly cloudy,0.0,88,6,1009,2019-01-06 22:00:00+00:00


FEATURE ENGINEERING

Merge Route_weather with Schedule data * create new data frame from scheduledf and add estimated_arrival and departure_date columns estimated_arriaval= df[estimated_arrival].dt.ceil("6H') departure_date= df[departure_date].dt.floor("6H")

In [88]:
truck_schedule.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10292 entries, 0 to 10291
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   truck_id           10292 non-null  int64 
 1   route_id           10292 non-null  object
 2   departure_date     10292 non-null  object
 3   estimated_arrival  10292 non-null  object
 4   delay              10292 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 402.2+ KB


In [89]:
routes_weather

,route_id,date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,R-df8c8ec9,2019-02-10 00:00:00+00:00,70,6,Sunny,0.0,62,6,1015
1,R-38d1e82d,2019-01-20 06:00:00+00:00,19,4,Clear,0.0,88,6,1021
2,R-593b486f,2019-02-13 18:00:00+00:00,63,3,Clear,0.0,88,6,1022
3,R-89580831,2019-01-29 06:00:00+00:00,68,2,Sunny,0.0,44,6,1017
4,R-da652dd4,2019-02-07 18:00:00+00:00,70,10,Sunny,0.0,46,6,1018
...,...,...,...,...,...,...,...,...,...
306564,R-af3e42c5,2019-02-12 18:00:00+00:00,70,9,Clear,0.0,45,6,1011
306565,R-513fb13a,2019-01-19 00:00:00+00:00,46,11,Clear,0.0,79,6,1015
306566,R-7ce7768c,2019-02-07 06:00:00+00:00,37,11,Clear,0.0,85,6,1008
306567,R-6f250521,2019-02-07 00:00:00+00:00,55,3,Clear,0.0,88,6,1016


Merge the resultant dataframe with route_weather on route_id and date (left)

In [90]:
import pandas as pd

def process_dates(df, estimated_arrival_column, departure_date_column):
    # Function to convert to UTC if not already timezone-aware
    def convert_to_utc(series):
        if series.dt.tz is None:  # Check if the series is not timezone-aware
            return series.dt.tz_localize('UTC')
        else:
            return series.dt.tz_convert('UTC')  # Convert to UTC if it already has a timezone

    # Step 1: Create a copy of the DataFrame to avoid modifying the original
    df_copy = df.copy()

    # Step 2: Convert both columns to datetime format and set to UTC timezone
    df_copy[estimated_arrival_column] = convert_to_utc(pd.to_datetime(df_copy[estimated_arrival_column], errors='coerce'))
    df_copy[departure_date_column] = convert_to_utc(pd.to_datetime(df_copy[departure_date_column], errors='coerce'))

    # Step 3: Apply ceil operation to estimated arrival
    df_copy[estimated_arrival_column] = df_copy[estimated_arrival_column].dt.ceil("6H")

    # Step 4: Floor the 'departure_date' column to the nearest 6 hours
    df_copy[departure_date_column] = df_copy[departure_date_column].dt.floor("6H")

    # Step 5: Create a new column 'date' with date ranges between 'departure_date' and 'estimated_arrival'
    df_copy['date'] = [
        pd.date_range(start=row[departure_date_column], end=row[estimated_arrival_column], freq='6H')
        for index, row in df_copy.iterrows()
    ]
    
    # Step 6: Explode the 'date' column to separate rows for each date in the range
    df_copy = df_copy.explode('date').reset_index(drop=True)

    return df_copy


# Define target tables
target_tables = ['truck_schedule']
departure_date_column = 'departure_date'
estimated_arrival_column = 'estimated_arrival'

# Process the DataFrames
for table_name in list(dfs.keys()):  # Use list(dfs.keys()) to avoid changing the dictionary during iteration
    df = dfs[table_name]
    if table_name in target_tables:
        # Create a copy and process dates
        result_df_copy = process_dates(df, estimated_arrival_column, departure_date_column)
        
        # Merge with route_weather on 'route_id' and 'date'
        scheduled_weather = pd.merge(result_df_copy, routes_weather, on=['route_id', 'date'], how='left')

        # Display the merged DataFrame
        print(scheduled_weather)

        # Save the processed DataFrame under a new key
        dfs[f'{table_name}_processed'] = result_df_copy


       truck_id    route_id            departure_date  \
0      46155960  R-c3d9f83b 2019-02-03 06:00:00+00:00   
1      46155960  R-c3d9f83b 2019-02-03 06:00:00+00:00   
2      23024395  R-f782724c 2019-01-10 06:00:00+00:00   
3      23024395  R-f782724c 2019-01-10 06:00:00+00:00   
4      23608280  R-4d09a561 2019-01-17 06:00:00+00:00   
...         ...         ...                       ...   
59674  11191961  R-8c2f0c3a 2019-01-25 06:00:00+00:00   
59675  17260588  R-a00ff8f8 2019-01-28 06:00:00+00:00   
59676  17260588  R-a00ff8f8 2019-01-28 06:00:00+00:00   
59677  17260588  R-a00ff8f8 2019-01-28 06:00:00+00:00   
59678  17260588  R-a00ff8f8 2019-01-28 06:00:00+00:00   

              estimated_arrival  delay                      date  temp  \
0     2019-02-03 12:00:00+00:00      1 2019-02-03 06:00:00+00:00  28.0   
1     2019-02-03 12:00:00+00:00      1 2019-02-03 12:00:00+00:00  61.0   
2     2019-01-10 12:00:00+00:00      1 2019-01-10 06:00:00+00:00  54.0   
3     2019-01-10 12

In [91]:
scheduled_weather

,truck_id,route_id,departure_date,estimated_arrival,delay,date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,46155960,R-c3d9f83b,2019-02-03 06:00:00+00:00,2019-02-03 12:00:00+00:00,1,2019-02-03 06:00:00+00:00,28.0,4.0,Clear,0.0,87.0,6.0,1005.0
1,46155960,R-c3d9f83b,2019-02-03 06:00:00+00:00,2019-02-03 12:00:00+00:00,1,2019-02-03 12:00:00+00:00,61.0,4.0,Clear,0.0,85.0,6.0,1012.0
2,23024395,R-f782724c,2019-01-10 06:00:00+00:00,2019-01-10 12:00:00+00:00,1,2019-01-10 06:00:00+00:00,54.0,4.0,Partly cloudy,0.0,58.0,6.0,1015.0
3,23024395,R-f782724c,2019-01-10 06:00:00+00:00,2019-01-10 12:00:00+00:00,1,2019-01-10 12:00:00+00:00,86.0,6.0,Sunny,0.0,62.0,6.0,1014.0
4,23608280,R-4d09a561,2019-01-17 06:00:00+00:00,2019-01-19 18:00:00+00:00,0,2019-01-17 06:00:00+00:00,63.0,10.0,Overcast,0.0,54.0,6.0,1018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59674,11191961,R-8c2f0c3a,2019-01-25 06:00:00+00:00,2019-01-26 00:00:00+00:00,0,2019-01-26 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59675,17260588,R-a00ff8f8,2019-01-28 06:00:00+00:00,2019-01-29 00:00:00+00:00,1,2019-01-28 06:00:00+00:00,84.0,9.0,Patchy rain possible,0.0,64.0,6.0,1015.0
59676,17260588,R-a00ff8f8,2019-01-28 06:00:00+00:00,2019-01-29 00:00:00+00:00,1,2019-01-28 12:00:00+00:00,79.0,8.0,Sunny,0.0,65.0,6.0,1007.0
59677,17260588,R-a00ff8f8,2019-01-28 06:00:00+00:00,2019-01-29 00:00:00+00:00,1,2019-01-28 18:00:00+00:00,77.0,6.0,Sunny,0.0,71.0,6.0,1007.0


In [92]:
import numpy as np
# Define a custom function to calculate mode with error handling
def custom_mode(x):
    mode_values = x.mode()
    if len(mode_values) > 0:
        return mode_values.iloc[0]
    else:
        return np.nan  # or a default value, such as '' or np.nan

# Group by specified columns and aggregate
scheduled_weather_grp = scheduled_weather.groupby(['truck_id','route_id'], as_index=False).agg(
             route_avg_temp=('temp','mean'),
             route_avg_wind_speed=('wind_speed','mean'),
             route_avg_precip=('precip','mean'),
             route_avg_humidity=('humidity','mean'),
             route_avg_visibility=('visibility','mean'),
             route_avg_pressure=('pressure','mean'),
             route_description=('description', custom_mode)
         )

scheduled_weather_grp

,truck_id,route_id,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,route_avg_pressure,route_description
0,10008392,R-06dfe39e,51.818182,6.545455,0.0,78.545455,6.0,1015.363636,Partly cloudy
1,10008392,R-35dca729,50.000000,10.875000,0.0,72.000000,6.0,1013.625000,Partly cloudy
2,10008392,R-51d1a32a,46.636364,8.909091,0.0,76.181818,6.0,1016.818182,Partly cloudy
3,10008392,R-7ab5ae85,85.500000,9.250000,0.0,67.250000,6.0,1009.500000,Sunny
4,10008392,R-c6f8e036,49.900000,5.600000,0.0,86.800000,6.0,1011.700000,Clear
...,...,...,...,...,...,...,...,...,...
10229,54866249,R-2f7cdfa5,58.578947,12.000000,0.0,60.526316,6.0,1014.263158,Sunny
10230,54866249,R-43613017,52.916667,5.708333,0.0,77.541667,6.0,1015.083333,Clear
10231,54866249,R-e3ad4c7e,85.769231,11.615385,0.0,67.846154,6.0,1013.384615,Sunny
10232,55417825,R-92744e22,71.121212,6.454545,0.0,70.454545,6.0,1015.575758,Patchy rain possible


merge schedule df with schedule_weather merge

In [93]:
schedule_weather_merge=truck_schedule.merge(scheduled_weather_grp,on=['truck_id','route_id'],how='left')
schedule_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,route_avg_pressure,route_description
0,46155960,R-c3d9f83b,2019-02-03 07:00:00,2019-02-03 08:31:48.000000000,1,44.500,4.000,0.0,86.000000,6.0,1008.500000,Clear
1,23024395,R-f782724c,2019-01-10 07:00:00,2019-01-10 11:55:48.000000000,1,70.000,5.000,0.0,60.000000,6.0,1014.500000,Partly cloudy
2,23608280,R-4d09a561,2019-01-17 07:00:00,2019-01-19 14:35:24.000000000,0,58.000,9.000,0.0,59.600000,6.0,1019.600000,Overcast
3,28641704,R-a55547f0,2019-01-16 07:00:00,2019-01-16 19:36:00.000000000,1,71.000,6.000,0.0,83.500000,6.0,1017.500000,Patchy rain possible
4,13426754,R-006548e0,2019-02-10 07:00:00,2019-02-11 23:01:12.000000000,1,51.875,4.875,0.0,72.500000,6.0,1022.000000,Partly cloudy
...,...,...,...,...,...,...,...,...,...,...,...,...
10287,28973978,R-68283907,2019-01-10 07:00:00,2019-01-11 03:36:00.000000000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10288,14969134,R-31ac88d8,2019-01-26 07:00:00,2019-01-27 18:10:12.000000000,1,71.200,5.000,0.0,70.000000,6.0,1014.400000,Sunny
10289,11699550,R-8ff40b97,2019-01-04 07:00:00,2019-01-04 16:25:12.000000000,1,62.000,5.000,0.0,68.333333,6.0,1018.333333,Sunny
10290,11191961,R-8c2f0c3a,2019-01-25 07:00:00,2019-01-25 21:06:36.000000000,0,72.000,13.500,0.0,41.500000,6.0,1014.000000,Sunny


Find origin and Destination city weather

In [94]:
#take hourly as weather data available hourly

import pandas as pd

# Create a copy of the truck_schedule_table DataFrame
nearest_hour_schedule_df = truck_schedule.copy()

# Convert 'estimated_arrival' and 'departure_date' columns to datetime (if they aren't already)
nearest_hour_schedule_df['estimated_arrival'] = pd.to_datetime(nearest_hour_schedule_df['estimated_arrival'], errors='coerce')
nearest_hour_schedule_df['departure_date'] = pd.to_datetime(nearest_hour_schedule_df['departure_date'], errors='coerce')

# Round 'estimated_arrival' to the nearest hour
nearest_hour_schedule_df['estimated_arrival_nearest_hour'] = nearest_hour_schedule_df['estimated_arrival'].dt.round("H")

# Round 'departure_date' to the nearest hour
nearest_hour_schedule_df['departure_date_nearest_hour'] = nearest_hour_schedule_df['departure_date'].dt.round("H")

# Merge the result with routes_table on 'route_id'
nearest_hour_schedule_route_df = pd.merge(nearest_hour_schedule_df, routes, on='route_id', how='left')

# Display the resulting DataFrame
print(nearest_hour_schedule_route_df)


       truck_id    route_id      departure_date   estimated_arrival  delay  \
0      46155960  R-c3d9f83b 2019-02-03 07:00:00 2019-02-03 08:31:48      1   
1      23024395  R-f782724c 2019-01-10 07:00:00 2019-01-10 11:55:48      1   
2      23608280  R-4d09a561 2019-01-17 07:00:00 2019-01-19 14:35:24      0   
3      28641704  R-a55547f0 2019-01-16 07:00:00 2019-01-16 19:36:00      1   
4      13426754  R-006548e0 2019-02-10 07:00:00 2019-02-11 23:01:12      1   
...         ...         ...                 ...                 ...    ...   
10287  28973978  R-68283907 2019-01-10 07:00:00 2019-01-11 03:36:00      0   
10288  14969134  R-31ac88d8 2019-01-26 07:00:00 2019-01-27 18:10:12      1   
10289  11699550  R-8ff40b97 2019-01-04 07:00:00 2019-01-04 16:25:12      1   
10290  11191961  R-8c2f0c3a 2019-01-25 07:00:00 2019-01-25 21:06:36      0   
10291  17260588  R-a00ff8f8 2019-01-28 07:00:00 2019-01-28 18:05:24      1   

      estimated_arrival_nearest_hour departure_date_nearest_hou

In [95]:
# Assume 'city_weather' is your original DataFrame
origin_weather_data= city_weather.copy()  # First copy
destination_weather_data = city_weather.copy()  # Second copy

In [96]:
nearest_hour_schedule_route_df.head(5)

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours
0,46155960,R-c3d9f83b,2019-02-03 07:00:00,2019-02-03 08:31:48,1,2019-02-03 09:00:00,2019-02-03 07:00:00,C-fc66f0ab,C-a9f2c329,76.48,1.53
1,23024395,R-f782724c,2019-01-10 07:00:00,2019-01-10 11:55:48,1,2019-01-10 12:00:00,2019-01-10 07:00:00,C-d9e9d934,C-b25a09de,246.53,4.93
2,23608280,R-4d09a561,2019-01-17 07:00:00,2019-01-19 14:35:24,0,2019-01-19 15:00:00,2019-01-17 07:00:00,C-4fe0fa24,C-f8f01604,2779.33,55.59
3,28641704,R-a55547f0,2019-01-16 07:00:00,2019-01-16 19:36:00,1,2019-01-16 20:00:00,2019-01-16 07:00:00,C-594514f8,C-84f378bb,629.90,12.60
4,13426754,R-006548e0,2019-02-10 07:00:00,2019-02-11 23:01:12,1,2019-02-11 23:00:00,2019-02-10 07:00:00,C-280b55fb,C-3dbd8b2e,2000.80,40.02


In [97]:
origin_weather_data

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,custom_date
0,C-19236709,55,10,Sunny,0.0,62,6,1001,2019-02-14 13:00:00+00:00
1,C-e6dcda92,19,17,Sunny,0.0,74,6,1027,2019-01-24 11:00:00+00:00
2,C-419cd14c,57,9,Sunny,0.0,70,6,1029,2019-01-09 06:00:00+00:00
3,C-ef47bdcd,36,7,Partly cloudy,0.0,53,6,1005,2019-01-06 07:00:00+00:00
4,C-34ae77b0,79,6,Clear,0.0,45,6,1016,2019-01-12 20:00:00+00:00
...,...,...,...,...,...,...,...,...,...
38835,C-2aaf0e1a,23,3,Patchy light snow,0.0,61,6,1027,2019-01-13 23:00:00+00:00
38836,C-b6e04c88,30,6,Patchy light snow,0.0,87,6,1019,2019-01-26 22:00:00+00:00
38837,C-c7cacd1d,52,11,Sunny,0.0,79,6,1011,2019-02-04 16:00:00+00:00
38838,C-2c9e75ef,57,17,Partly cloudy,0.0,88,6,1009,2019-01-06 22:00:00+00:00


            * create a copy of the 'weather_df' DataFrame for manipulation
            * Drop the 'date' and 'hour' columns from 'origin_weather_data'
            * Create a copy of the 'weather_df' DataFrame for manipulation
            * Drop the 'date' and 'hour' columns from 'destination_weather_data'
            * Merge 'nearest_hour_schedule_route_df' with 'origin_weather_data' based on specified columns


In [98]:
#Merge 'nearest_hour_schedule_route_df' with 'origin_weather_data' based on specified columns

# Ensure both DataFrames have the datetime columns in UTC format

# Check if departure_date_nearest_hour is timezone-naive
if nearest_hour_schedule_route_df['departure_date_nearest_hour'].dt.tz is None:
    # Localize to UTC if naive
    nearest_hour_schedule_route_df['departure_date_nearest_hour'] = nearest_hour_schedule_route_df['departure_date_nearest_hour'].dt.tz_localize('UTC')

# Check if custom_date is timezone-naive
if origin_weather_data['custom_date'].dt.tz is None:
    # Localize to UTC if naive
    origin_weather_data['custom_date'] = pd.to_datetime(origin_weather_data['custom_date'], errors='coerce').dt.tz_localize('UTC')

# Specify the columns to merge on
left_merge_columns = ['origin_id', 'departure_date_nearest_hour']  # Columns from nearest_hour_schedule_route_df
right_merge_columns = ['city_id', 'custom_date']  # Columns from origin_weather_data

# Merge the DataFrames
origin_weather_merge = pd.merge(
    nearest_hour_schedule_route_df, 
    origin_weather_data, 
    left_on=left_merge_columns, 
    right_on=right_merge_columns, 
    how='left'
)

# Display the resulting merged DataFrame
origin_weather_merge


,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,custom_date
0,46155960,R-c3d9f83b,2019-02-03 07:00:00,2019-02-03 08:31:48,1,2019-02-03 09:00:00,2019-02-03 07:00:00+00:00,C-fc66f0ab,C-a9f2c329,76.48,1.53,C-fc66f0ab,39.0,9.0,Overcast,0.0,100.0,6.0,1012.0,2019-02-03 07:00:00+00:00
1,23024395,R-f782724c,2019-01-10 07:00:00,2019-01-10 11:55:48,1,2019-01-10 12:00:00,2019-01-10 07:00:00+00:00,C-d9e9d934,C-b25a09de,246.53,4.93,C-d9e9d934,32.0,16.0,Sunny,0.0,55.0,6.0,1017.0,2019-01-10 07:00:00+00:00
2,23608280,R-4d09a561,2019-01-17 07:00:00,2019-01-19 14:35:24,0,2019-01-19 15:00:00,2019-01-17 07:00:00+00:00,C-4fe0fa24,C-f8f01604,2779.33,55.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,28641704,R-a55547f0,2019-01-16 07:00:00,2019-01-16 19:36:00,1,2019-01-16 20:00:00,2019-01-16 07:00:00+00:00,C-594514f8,C-84f378bb,629.90,12.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,13426754,R-006548e0,2019-02-10 07:00:00,2019-02-11 23:01:12,1,2019-02-11 23:00:00,2019-02-10 07:00:00+00:00,C-280b55fb,C-3dbd8b2e,2000.80,40.02,C-280b55fb,21.0,6.0,Sunny,0.0,50.0,6.0,1036.0,2019-02-10 07:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,28973978,R-68283907,2019-01-10 07:00:00,2019-01-11 03:36:00,0,2019-01-11 04:00:00,2019-01-10 07:00:00+00:00,C-841ebdcb,C-280b55fb,1029.88,20.60,C-841ebdcb,54.0,11.0,Sunny,0.0,52.0,6.0,1028.0,2019-01-10 07:00:00+00:00
10288,14969134,R-31ac88d8,2019-01-26 07:00:00,2019-01-27 18:10:12,1,2019-01-27 18:00:00,2019-01-26 07:00:00+00:00,C-328bd8d3,C-56e39a5e,1758.52,35.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
10289,11699550,R-8ff40b97,2019-01-04 07:00:00,2019-01-04 16:25:12,1,2019-01-04 16:00:00,2019-01-04 07:00:00+00:00,C-2aaf0e1a,C-594514f8,470.98,9.42,C-2aaf0e1a,32.0,4.0,Sunny,0.0,79.0,6.0,1016.0,2019-01-04 07:00:00+00:00
10290,11191961,R-8c2f0c3a,2019-01-25 07:00:00,2019-01-25 21:06:36,0,2019-01-25 21:00:00,2019-01-25 07:00:00+00:00,C-b6e04c88,C-c7cacd1d,705.50,14.11,C-b6e04c88,12.0,6.0,Sunny,0.0,57.0,6.0,1030.0,2019-01-25 07:00:00+00:00


In [99]:
destination_weather_data

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,custom_date
0,C-19236709,55,10,Sunny,0.0,62,6,1001,2019-02-14 13:00:00+00:00
1,C-e6dcda92,19,17,Sunny,0.0,74,6,1027,2019-01-24 11:00:00+00:00
2,C-419cd14c,57,9,Sunny,0.0,70,6,1029,2019-01-09 06:00:00+00:00
3,C-ef47bdcd,36,7,Partly cloudy,0.0,53,6,1005,2019-01-06 07:00:00+00:00
4,C-34ae77b0,79,6,Clear,0.0,45,6,1016,2019-01-12 20:00:00+00:00
...,...,...,...,...,...,...,...,...,...
38835,C-2aaf0e1a,23,3,Patchy light snow,0.0,61,6,1027,2019-01-13 23:00:00+00:00
38836,C-b6e04c88,30,6,Patchy light snow,0.0,87,6,1019,2019-01-26 22:00:00+00:00
38837,C-c7cacd1d,52,11,Sunny,0.0,79,6,1011,2019-02-04 16:00:00+00:00
38838,C-2c9e75ef,57,17,Partly cloudy,0.0,88,6,1009,2019-01-06 22:00:00+00:00


Merge 'origin_weather_merge' with 'destination_weather_data' based on specified columns


In [100]:
#Merge 'origin_weather_merge' with 'destination_weather_data' based on specified columns

# Ensure both DataFrames have the datetime columns in UTC format

# Check if estimated_arrival_nearest_hour is timezone-naive
if origin_weather_merge['estimated_arrival_nearest_hour'].dt.tz is None:
    # Localize to UTC if naive
    origin_weather_merge['estimated_arrival_nearest_hour'] = origin_weather_merge['estimated_arrival_nearest_hour'].dt.tz_localize('UTC')

# Check if custom_date is timezone-naive
if destination_weather_data['custom_date'].dt.tz is None:
    # Localize to UTC if naive
    destination_weather_data['custom_date'] = pd.to_datetime(destination_weather_data['custom_date'], errors='coerce').dt.tz_localize('UTC')

# Specify the columns to merge on
left_merge_columns = ['destination_id', 'estimated_arrival_nearest_hour']  # Columns from origin_weather_merge
right_merge_columns = ['city_id', 'custom_date']  # Columns from destination_weather_data

# Merge the DataFrames
schedule_data_merge = pd.merge(
    origin_weather_merge, 
    destination_weather_data, 
    left_on=left_merge_columns, 
    right_on=right_merge_columns, 
    how='left'
)

# Display the resulting merged DataFrame
schedule_data_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,custom_date_x,city_id_y,temp_y,wind_speed_y,description_y,precip_y,humidity_y,visibility_y,pressure_y,custom_date_y
0,46155960,R-c3d9f83b,2019-02-03 07:00:00,2019-02-03 08:31:48,1,2019-02-03 09:00:00+00:00,2019-02-03 07:00:00+00:00,C-fc66f0ab,C-a9f2c329,76.48,...,2019-02-03 07:00:00+00:00,C-a9f2c329,43.0,12.0,Overcast,0.0,98.0,6.0,1011.0,2019-02-03 09:00:00+00:00
1,23024395,R-f782724c,2019-01-10 07:00:00,2019-01-10 11:55:48,1,2019-01-10 12:00:00+00:00,2019-01-10 07:00:00+00:00,C-d9e9d934,C-b25a09de,246.53,...,2019-01-10 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,23608280,R-4d09a561,2019-01-17 07:00:00,2019-01-19 14:35:24,0,2019-01-19 15:00:00+00:00,2019-01-17 07:00:00+00:00,C-4fe0fa24,C-f8f01604,2779.33,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,28641704,R-a55547f0,2019-01-16 07:00:00,2019-01-16 19:36:00,1,2019-01-16 20:00:00+00:00,2019-01-16 07:00:00+00:00,C-594514f8,C-84f378bb,629.90,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,13426754,R-006548e0,2019-02-10 07:00:00,2019-02-11 23:01:12,1,2019-02-11 23:00:00+00:00,2019-02-10 07:00:00+00:00,C-280b55fb,C-3dbd8b2e,2000.80,...,2019-02-10 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,28973978,R-68283907,2019-01-10 07:00:00,2019-01-11 03:36:00,0,2019-01-11 04:00:00+00:00,2019-01-10 07:00:00+00:00,C-841ebdcb,C-280b55fb,1029.88,...,2019-01-10 07:00:00+00:00,C-280b55fb,25.0,7.0,Clear,0.0,59.0,6.0,1025.0,2019-01-11 04:00:00+00:00
10288,14969134,R-31ac88d8,2019-01-26 07:00:00,2019-01-27 18:10:12,1,2019-01-27 18:00:00+00:00,2019-01-26 07:00:00+00:00,C-328bd8d3,C-56e39a5e,1758.52,...,NaT,C-56e39a5e,73.0,9.0,Patchy rain possible,0.0,72.0,6.0,1015.0,2019-01-27 18:00:00+00:00
10289,11699550,R-8ff40b97,2019-01-04 07:00:00,2019-01-04 16:25:12,1,2019-01-04 16:00:00+00:00,2019-01-04 07:00:00+00:00,C-2aaf0e1a,C-594514f8,470.98,...,2019-01-04 07:00:00+00:00,C-594514f8,41.0,2.0,Sunny,0.0,68.0,6.0,1007.0,2019-01-04 16:00:00+00:00
10290,11191961,R-8c2f0c3a,2019-01-25 07:00:00,2019-01-25 21:06:36,0,2019-01-25 21:00:00+00:00,2019-01-25 07:00:00+00:00,C-b6e04c88,C-c7cacd1d,705.50,...,2019-01-25 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


Traffic and schedule data merge

In [101]:
# Create a copy of the schedule_data_merge DataFrame
schedule_data_copy = schedule_data_merge.copy()
schedule_data_copy

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,custom_date_x,city_id_y,temp_y,wind_speed_y,description_y,precip_y,humidity_y,visibility_y,pressure_y,custom_date_y
0,46155960,R-c3d9f83b,2019-02-03 07:00:00,2019-02-03 08:31:48,1,2019-02-03 09:00:00+00:00,2019-02-03 07:00:00+00:00,C-fc66f0ab,C-a9f2c329,76.48,...,2019-02-03 07:00:00+00:00,C-a9f2c329,43.0,12.0,Overcast,0.0,98.0,6.0,1011.0,2019-02-03 09:00:00+00:00
1,23024395,R-f782724c,2019-01-10 07:00:00,2019-01-10 11:55:48,1,2019-01-10 12:00:00+00:00,2019-01-10 07:00:00+00:00,C-d9e9d934,C-b25a09de,246.53,...,2019-01-10 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,23608280,R-4d09a561,2019-01-17 07:00:00,2019-01-19 14:35:24,0,2019-01-19 15:00:00+00:00,2019-01-17 07:00:00+00:00,C-4fe0fa24,C-f8f01604,2779.33,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,28641704,R-a55547f0,2019-01-16 07:00:00,2019-01-16 19:36:00,1,2019-01-16 20:00:00+00:00,2019-01-16 07:00:00+00:00,C-594514f8,C-84f378bb,629.90,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,13426754,R-006548e0,2019-02-10 07:00:00,2019-02-11 23:01:12,1,2019-02-11 23:00:00+00:00,2019-02-10 07:00:00+00:00,C-280b55fb,C-3dbd8b2e,2000.80,...,2019-02-10 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,28973978,R-68283907,2019-01-10 07:00:00,2019-01-11 03:36:00,0,2019-01-11 04:00:00+00:00,2019-01-10 07:00:00+00:00,C-841ebdcb,C-280b55fb,1029.88,...,2019-01-10 07:00:00+00:00,C-280b55fb,25.0,7.0,Clear,0.0,59.0,6.0,1025.0,2019-01-11 04:00:00+00:00
10288,14969134,R-31ac88d8,2019-01-26 07:00:00,2019-01-27 18:10:12,1,2019-01-27 18:00:00+00:00,2019-01-26 07:00:00+00:00,C-328bd8d3,C-56e39a5e,1758.52,...,NaT,C-56e39a5e,73.0,9.0,Patchy rain possible,0.0,72.0,6.0,1015.0,2019-01-27 18:00:00+00:00
10289,11699550,R-8ff40b97,2019-01-04 07:00:00,2019-01-04 16:25:12,1,2019-01-04 16:00:00+00:00,2019-01-04 07:00:00+00:00,C-2aaf0e1a,C-594514f8,470.98,...,2019-01-04 07:00:00+00:00,C-594514f8,41.0,2.0,Sunny,0.0,68.0,6.0,1007.0,2019-01-04 16:00:00+00:00
10290,11191961,R-8c2f0c3a,2019-01-25 07:00:00,2019-01-25 21:06:36,0,2019-01-25 21:00:00+00:00,2019-01-25 07:00:00+00:00,C-b6e04c88,C-c7cacd1d,705.50,...,2019-01-25 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


Round 'estimated_arrival' times to the nearest hour
    * Round 'departure_date' times to the nearest hour


In [102]:
# Round 'estimated_arrival' to the nearest hour
schedule_data_copy['estimated_arrival'] = schedule_data_copy['estimated_arrival'].dt.round('H')

# Round 'departure_date' to the nearest hour
schedule_data_copy['departure_date'] = schedule_data_copy['departure_date'].dt.round('H')

# Display the modified DataFrame
schedule_data_copy[['estimated_arrival', 'departure_date']].head()

nearest_hour_schedule_df = schedule_data_copy
nearest_hour_schedule_df


,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,custom_date_x,city_id_y,temp_y,wind_speed_y,description_y,precip_y,humidity_y,visibility_y,pressure_y,custom_date_y
0,46155960,R-c3d9f83b,2019-02-03 07:00:00,2019-02-03 09:00:00,1,2019-02-03 09:00:00+00:00,2019-02-03 07:00:00+00:00,C-fc66f0ab,C-a9f2c329,76.48,...,2019-02-03 07:00:00+00:00,C-a9f2c329,43.0,12.0,Overcast,0.0,98.0,6.0,1011.0,2019-02-03 09:00:00+00:00
1,23024395,R-f782724c,2019-01-10 07:00:00,2019-01-10 12:00:00,1,2019-01-10 12:00:00+00:00,2019-01-10 07:00:00+00:00,C-d9e9d934,C-b25a09de,246.53,...,2019-01-10 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,23608280,R-4d09a561,2019-01-17 07:00:00,2019-01-19 15:00:00,0,2019-01-19 15:00:00+00:00,2019-01-17 07:00:00+00:00,C-4fe0fa24,C-f8f01604,2779.33,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,28641704,R-a55547f0,2019-01-16 07:00:00,2019-01-16 20:00:00,1,2019-01-16 20:00:00+00:00,2019-01-16 07:00:00+00:00,C-594514f8,C-84f378bb,629.90,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,13426754,R-006548e0,2019-02-10 07:00:00,2019-02-11 23:00:00,1,2019-02-11 23:00:00+00:00,2019-02-10 07:00:00+00:00,C-280b55fb,C-3dbd8b2e,2000.80,...,2019-02-10 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,28973978,R-68283907,2019-01-10 07:00:00,2019-01-11 04:00:00,0,2019-01-11 04:00:00+00:00,2019-01-10 07:00:00+00:00,C-841ebdcb,C-280b55fb,1029.88,...,2019-01-10 07:00:00+00:00,C-280b55fb,25.0,7.0,Clear,0.0,59.0,6.0,1025.0,2019-01-11 04:00:00+00:00
10288,14969134,R-31ac88d8,2019-01-26 07:00:00,2019-01-27 18:00:00,1,2019-01-27 18:00:00+00:00,2019-01-26 07:00:00+00:00,C-328bd8d3,C-56e39a5e,1758.52,...,NaT,C-56e39a5e,73.0,9.0,Patchy rain possible,0.0,72.0,6.0,1015.0,2019-01-27 18:00:00+00:00
10289,11699550,R-8ff40b97,2019-01-04 07:00:00,2019-01-04 16:00:00,1,2019-01-04 16:00:00+00:00,2019-01-04 07:00:00+00:00,C-2aaf0e1a,C-594514f8,470.98,...,2019-01-04 07:00:00+00:00,C-594514f8,41.0,2.0,Sunny,0.0,68.0,6.0,1007.0,2019-01-04 16:00:00+00:00
10290,11191961,R-8c2f0c3a,2019-01-25 07:00:00,2019-01-25 21:00:00,0,2019-01-25 21:00:00+00:00,2019-01-25 07:00:00+00:00,C-b6e04c88,C-c7cacd1d,705.50,...,2019-01-25 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [103]:
# Function to ensure datetime columns are in UTC
def ensure_utc(column):
    if column.dt.tz is None:  # Check if the column is naive
        return column.dt.tz_localize('UTC')  # Localize to UTC
    else:
        return column.dt.tz_convert('UTC')  # Convert to UTC if already aware

# Ensure both datetime columns are in UTC
nearest_hour_schedule_df['departure_date'] = ensure_utc(nearest_hour_schedule_df['departure_date'])
nearest_hour_schedule_df['estimated_arrival'] = ensure_utc(nearest_hour_schedule_df['estimated_arrival'])

# Now create the custom date ranges
hourly_exploded_scheduled_df = (
    nearest_hour_schedule_df.assign(
        custom_date=[
            pd.date_range(start, end, freq='H')  # Create custom date ranges
            for start, end in zip(
                nearest_hour_schedule_df['departure_date'],
                nearest_hour_schedule_df['estimated_arrival']
            )  # Using departure and estimated arrival times
        ]
    ).explode('custom_date', ignore_index=True)  # Explode the DataFrame based on the custom date range
)

# Display the resulting DataFrame
hourly_exploded_scheduled_df[['departure_date', 'estimated_arrival', 'custom_date']].head()


,departure_date,estimated_arrival,custom_date
0,2019-02-03 07:00:00+00:00,2019-02-03 09:00:00+00:00,2019-02-03 07:00:00+00:00
1,2019-02-03 07:00:00+00:00,2019-02-03 09:00:00+00:00,2019-02-03 08:00:00+00:00
2,2019-02-03 07:00:00+00:00,2019-02-03 09:00:00+00:00,2019-02-03 09:00:00+00:00
3,2019-01-10 07:00:00+00:00,2019-01-10 12:00:00+00:00,2019-01-10 07:00:00+00:00
4,2019-01-10 07:00:00+00:00,2019-01-10 12:00:00+00:00,2019-01-10 08:00:00+00:00


In [104]:
traffic

,route_id,no_of_vehicles,accident,datetime
0,R-073654b1,501.0,0,2019-01-01 08:00:00+00:00
1,R-0484f127,631.0,0,2019-01-03 02:00:00+00:00
2,R-ad41fea2,1890.0,0,2019-02-06 16:00:00+00:00
3,R-8dd113f2,604.0,0,2019-01-12 08:00:00+00:00
4,R-a6c7b143,2164.0,0,2019-02-08 11:00:00+00:00
...,...,...,...,...
2458577,R-20612873,699.0,0,2019-01-23 02:00:00+00:00
2458578,R-d92c1aa1,619.0,0,2019-01-24 11:00:00+00:00
2458579,R-1a08406e,1885.0,0,2019-02-11 16:00:00+00:00
2458580,R-7e29ed2b,2536.0,0,2019-01-25 10:00:00+00:00


In [105]:
hourly_exploded_scheduled_df

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,city_id_y,temp_y,wind_speed_y,description_y,precip_y,humidity_y,visibility_y,pressure_y,custom_date_y,custom_date
0,46155960,R-c3d9f83b,2019-02-03 07:00:00+00:00,2019-02-03 09:00:00+00:00,1,2019-02-03 09:00:00+00:00,2019-02-03 07:00:00+00:00,C-fc66f0ab,C-a9f2c329,76.48,...,C-a9f2c329,43.0,12.0,Overcast,0.0,98.0,6.0,1011.0,2019-02-03 09:00:00+00:00,2019-02-03 07:00:00+00:00
1,46155960,R-c3d9f83b,2019-02-03 07:00:00+00:00,2019-02-03 09:00:00+00:00,1,2019-02-03 09:00:00+00:00,2019-02-03 07:00:00+00:00,C-fc66f0ab,C-a9f2c329,76.48,...,C-a9f2c329,43.0,12.0,Overcast,0.0,98.0,6.0,1011.0,2019-02-03 09:00:00+00:00,2019-02-03 08:00:00+00:00
2,46155960,R-c3d9f83b,2019-02-03 07:00:00+00:00,2019-02-03 09:00:00+00:00,1,2019-02-03 09:00:00+00:00,2019-02-03 07:00:00+00:00,C-fc66f0ab,C-a9f2c329,76.48,...,C-a9f2c329,43.0,12.0,Overcast,0.0,98.0,6.0,1011.0,2019-02-03 09:00:00+00:00,2019-02-03 09:00:00+00:00
3,23024395,R-f782724c,2019-01-10 07:00:00+00:00,2019-01-10 12:00:00+00:00,1,2019-01-10 12:00:00+00:00,2019-01-10 07:00:00+00:00,C-d9e9d934,C-b25a09de,246.53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2019-01-10 07:00:00+00:00
4,23024395,R-f782724c,2019-01-10 07:00:00+00:00,2019-01-10 12:00:00+00:00,1,2019-01-10 12:00:00+00:00,2019-01-10 07:00:00+00:00,C-d9e9d934,C-b25a09de,246.53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2019-01-10 08:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265410,17260588,R-a00ff8f8,2019-01-28 07:00:00+00:00,2019-01-28 18:00:00+00:00,1,2019-01-28 18:00:00+00:00,2019-01-28 07:00:00+00:00,C-03bb3e48,C-594514f8,554.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2019-01-28 14:00:00+00:00
265411,17260588,R-a00ff8f8,2019-01-28 07:00:00+00:00,2019-01-28 18:00:00+00:00,1,2019-01-28 18:00:00+00:00,2019-01-28 07:00:00+00:00,C-03bb3e48,C-594514f8,554.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2019-01-28 15:00:00+00:00
265412,17260588,R-a00ff8f8,2019-01-28 07:00:00+00:00,2019-01-28 18:00:00+00:00,1,2019-01-28 18:00:00+00:00,2019-01-28 07:00:00+00:00,C-03bb3e48,C-594514f8,554.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2019-01-28 16:00:00+00:00
265413,17260588,R-a00ff8f8,2019-01-28 07:00:00+00:00,2019-01-28 18:00:00+00:00,1,2019-01-28 18:00:00+00:00,2019-01-28 07:00:00+00:00,C-03bb3e48,C-594514f8,554.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2019-01-28 17:00:00+00:00


In [106]:
# Merge the traffic_table and hourly_exploded_scheduled_df DataFrames
scheduled_traffic = hourly_exploded_scheduled_df.merge(
    traffic,
    left_on=['route_id', 'custom_date'],  # Columns from hourly_exploded_scheduled_df
    right_on=['route_id', 'datetime'],        # Corresponding columns from traffic_table
    how='left'
)

# Display the resulting merged DataFrame
scheduled_traffic.head()


,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,description_y,precip_y,humidity_y,visibility_y,pressure_y,custom_date_y,custom_date,no_of_vehicles,accident,datetime
0,46155960,R-c3d9f83b,2019-02-03 07:00:00+00:00,2019-02-03 09:00:00+00:00,1,2019-02-03 09:00:00+00:00,2019-02-03 07:00:00+00:00,C-fc66f0ab,C-a9f2c329,76.48,...,Overcast,0.0,98.0,6.0,1011.0,2019-02-03 09:00:00+00:00,2019-02-03 07:00:00+00:00,2046.0,0.0,2019-02-03 07:00:00+00:00
1,46155960,R-c3d9f83b,2019-02-03 07:00:00+00:00,2019-02-03 09:00:00+00:00,1,2019-02-03 09:00:00+00:00,2019-02-03 07:00:00+00:00,C-fc66f0ab,C-a9f2c329,76.48,...,Overcast,0.0,98.0,6.0,1011.0,2019-02-03 09:00:00+00:00,2019-02-03 08:00:00+00:00,2376.0,0.0,2019-02-03 08:00:00+00:00
2,46155960,R-c3d9f83b,2019-02-03 07:00:00+00:00,2019-02-03 09:00:00+00:00,1,2019-02-03 09:00:00+00:00,2019-02-03 07:00:00+00:00,C-fc66f0ab,C-a9f2c329,76.48,...,Overcast,0.0,98.0,6.0,1011.0,2019-02-03 09:00:00+00:00,2019-02-03 09:00:00+00:00,2439.0,0.0,2019-02-03 09:00:00+00:00
3,23024395,R-f782724c,2019-01-10 07:00:00+00:00,2019-01-10 12:00:00+00:00,1,2019-01-10 12:00:00+00:00,2019-01-10 07:00:00+00:00,C-d9e9d934,C-b25a09de,246.53,...,NaN,NaN,NaN,NaN,NaN,NaT,2019-01-10 07:00:00+00:00,2523.0,0.0,2019-01-10 07:00:00+00:00
4,23024395,R-f782724c,2019-01-10 07:00:00+00:00,2019-01-10 12:00:00+00:00,1,2019-01-10 12:00:00+00:00,2019-01-10 07:00:00+00:00,C-d9e9d934,C-b25a09de,246.53,...,NaN,NaN,NaN,NaN,NaN,NaT,2019-01-10 08:00:00+00:00,2268.0,0.0,2019-01-10 08:00:00+00:00


In [107]:
#Define a custom aggregation function for accidents
def custom_agg(values):
    """
    Custom aggregation function to determine if any value in a group is 1 (indicating an accident).
    
    Args:
    values (iterable): Iterable of values in a group.
    
    Returns:
    int: 1 if any value is 1, else 0.
    """
    # Check if any value in the iterable is equal to 1
    return 1 if 1 in values else 0

# Group by 'unique_id', 'truck_id', and 'route_id', and apply custom aggregation
scheduled_route_traffic = scheduled_traffic.groupby([ 'truck_id', 'route_id'], as_index=False).agg(
    avg_no_of_vehicles=('no_of_vehicles', 'mean'),
    accident=('accident', custom_agg)
)

# Display the resulting DataFrame
scheduled_route_traffic.head()

,truck_id,route_id,avg_no_of_vehicles,accident
0,10008392,R-06dfe39e,1887.898305,0
1,10008392,R-35dca729,1833.734694,0
2,10008392,R-51d1a32a,1532.553571,0
3,10008392,R-7ab5ae85,1976.000000,0
4,10008392,R-c6f8e036,846.224138,0


Merge data

In [108]:
#renaming the schedule_data_merge with origin_destination_weather
origin_destination_weather=schedule_data_merge

In [109]:
#Merge schedule_route_traffic with origin_destination_weather

origin_destination_weather_traffic_merge=origin_destination_weather.merge(scheduled_route_traffic,on=['truck_id','route_id'],how='left')

Merge weather and Traffic

In [110]:
# Convert 'departure_date' to datetime in the schedule_weather_merge DataFrame
schedule_weather_merge['departure_date'] = pd.to_datetime(schedule_weather_merge['departure_date'], errors='coerce')

# If origin_destination_weather_traffic_merge has 'departure_date' as object, convert it as well
origin_destination_weather_traffic_merge['departure_date'] = pd.to_datetime(origin_destination_weather_traffic_merge['departure_date'], errors='coerce')


In [111]:
schedule_weather_merge.dtypes

truck_id                         int64
route_id                        object
departure_date          datetime64[ns]
estimated_arrival               object
delay                            int64
route_avg_temp                 float64
route_avg_wind_speed           float64
route_avg_precip               float64
route_avg_humidity             float64
route_avg_visibility           float64
route_avg_pressure             float64
route_description               object
dtype: object

In [112]:
schedule_weather_merge['estimated_arrival'] = pd.to_datetime(schedule_weather_merge['estimated_arrival'], errors='coerce')

In [113]:
schedule_weather_merge.dtypes

truck_id                         int64
route_id                        object
departure_date          datetime64[ns]
estimated_arrival       datetime64[ns]
delay                            int64
route_avg_temp                 float64
route_avg_wind_speed           float64
route_avg_precip               float64
route_avg_humidity             float64
route_avg_visibility           float64
route_avg_pressure             float64
route_description               object
dtype: object

In [114]:
origin_destination_weather_traffic_merge.dtypes

truck_id                                        int64
route_id                                       object
departure_date                         datetime64[ns]
estimated_arrival                      datetime64[ns]
delay                                           int64
estimated_arrival_nearest_hour    datetime64[ns, UTC]
departure_date_nearest_hour       datetime64[ns, UTC]
origin_id                                      object
destination_id                                 object
distance                                      float64
average_hours                                 float64
city_id_x                                      object
temp_x                                        float64
wind_speed_x                                  float64
description_x                                  object
precip_x                                      float64
humidity_x                                    float64
visibility_x                                  float64
pressure_x                  

In [115]:
#merged_data_weather_traffic=pd.merge(schedule_weather_merge, origin_destination_weather_traffic_merge, on=['truck_id', 'route_id', 'departure_date','estimated_arrival', 'delay'], how='left')

# Merge the DataFrames
merged_data_weather_traffic = pd.merge(
    schedule_weather_merge, 
    origin_destination_weather_traffic_merge, 
    on=['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay'], 
    how='left'
)

# Display the merged DataFrame
merged_data_weather_traffic.head()


,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,...,temp_y,wind_speed_y,description_y,precip_y,humidity_y,visibility_y,pressure_y,custom_date_y,avg_no_of_vehicles,accident
0,46155960,R-c3d9f83b,2019-02-03 07:00:00,2019-02-03 08:31:48,1,44.500,4.000,0.0,86.0,6.0,...,43.0,12.0,Overcast,0.0,98.0,6.0,1011.0,2019-02-03 09:00:00+00:00,2287.000000,1
1,23024395,R-f782724c,2019-01-10 07:00:00,2019-01-10 11:55:48,1,70.000,5.000,0.0,60.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2257.500000,0
2,23608280,R-4d09a561,2019-01-17 07:00:00,2019-01-19 14:35:24,0,58.000,9.000,0.0,59.6,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1831.181818,0
3,28641704,R-a55547f0,2019-01-16 07:00:00,2019-01-16 19:36:00,1,71.000,6.000,0.0,83.5,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2276.250000,0
4,13426754,R-006548e0,2019-02-10 07:00:00,2019-02-11 23:01:12,1,51.875,4.875,0.0,72.5,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,989.459459,0


In [116]:
merged_data_weather_traffic['estimated_arrival'] = merged_data_weather_traffic['estimated_arrival'].dt.ceil('6H')

In [117]:
merged_data_weather_traffic

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,...,temp_y,wind_speed_y,description_y,precip_y,humidity_y,visibility_y,pressure_y,custom_date_y,avg_no_of_vehicles,accident
0,46155960,R-c3d9f83b,2019-02-03 07:00:00,2019-02-03 12:00:00,1,44.500,4.000,0.0,86.000000,6.0,...,43.0,12.0,Overcast,0.0,98.0,6.0,1011.0,2019-02-03 09:00:00+00:00,2287.000000,1
1,23024395,R-f782724c,2019-01-10 07:00:00,2019-01-10 12:00:00,1,70.000,5.000,0.0,60.000000,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2257.500000,0
2,23608280,R-4d09a561,2019-01-17 07:00:00,2019-01-19 18:00:00,0,58.000,9.000,0.0,59.600000,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1831.181818,0
3,28641704,R-a55547f0,2019-01-16 07:00:00,2019-01-17 00:00:00,1,71.000,6.000,0.0,83.500000,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2276.250000,0
4,13426754,R-006548e0,2019-02-10 07:00:00,2019-02-12 00:00:00,1,51.875,4.875,0.0,72.500000,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,989.459459,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,28973978,R-68283907,2019-01-10 07:00:00,2019-01-11 06:00:00,0,NaN,NaN,NaN,NaN,NaN,...,25.0,7.0,Clear,0.0,59.0,6.0,1025.0,2019-01-11 04:00:00+00:00,1833.227273,0
10288,14969134,R-31ac88d8,2019-01-26 07:00:00,2019-01-28 00:00:00,1,71.200,5.000,0.0,70.000000,6.0,...,73.0,9.0,Patchy rain possible,0.0,72.0,6.0,1015.0,2019-01-27 18:00:00+00:00,1391.400000,0
10289,11699550,R-8ff40b97,2019-01-04 07:00:00,2019-01-04 18:00:00,1,62.000,5.000,0.0,68.333333,6.0,...,41.0,2.0,Sunny,0.0,68.0,6.0,1007.0,2019-01-04 16:00:00+00:00,2237.600000,0
10290,11191961,R-8c2f0c3a,2019-01-25 07:00:00,2019-01-26 00:00:00,0,72.000,13.500,0.0,41.500000,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2353.384615,0


In [118]:
merged_data_weather_traffic.isnull().sum()

truck_id                             0
route_id                             0
departure_date                       0
estimated_arrival                    0
delay                                0
route_avg_temp                     490
route_avg_wind_speed               490
route_avg_precip                   490
route_avg_humidity                 490
route_avg_visibility               490
route_avg_pressure                 490
route_description                  490
estimated_arrival_nearest_hour       0
departure_date_nearest_hour          0
origin_id                          711
destination_id                     711
distance                           711
average_hours                      711
city_id_x                         4054
temp_x                            4054
wind_speed_x                      4054
description_x                     4054
precip_x                          4054
humidity_x                        4054
visibility_x                      4054
pressure_x               

In [119]:
bswd;qjhdbq'skq ;jlshqjlsq;ksl  

SyntaxError: unterminated string literal (detected at line 1) (1172191745.py, line 1)

In [120]:
merged_data_weather_traffic = merged_data_weather_traffic.fillna(method='ffill')

2024-10-02 12:41:07,980 WARNING: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [121]:
merged_data_weather_traffic.isnull().sum()

truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
delay                             0
route_avg_temp                    0
route_avg_wind_speed              0
route_avg_precip                  0
route_avg_humidity                0
route_avg_visibility              0
route_avg_pressure                0
route_description                 0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
origin_id                         0
destination_id                    0
distance                          0
average_hours                     0
city_id_x                         0
temp_x                            0
wind_speed_x                      0
description_x                     0
precip_x                          0
humidity_x                        0
visibility_x                      0
pressure_x                        0
custom_date_x                     0
city_id_y                   

In [122]:
merged_data_weather_traffic

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,...,temp_y,wind_speed_y,description_y,precip_y,humidity_y,visibility_y,pressure_y,custom_date_y,avg_no_of_vehicles,accident
0,46155960,R-c3d9f83b,2019-02-03 07:00:00,2019-02-03 12:00:00,1,44.500,4.000,0.0,86.000000,6.0,...,43.0,12.0,Overcast,0.0,98.0,6.0,1011.0,2019-02-03 09:00:00+00:00,2287.000000,1
1,23024395,R-f782724c,2019-01-10 07:00:00,2019-01-10 12:00:00,1,70.000,5.000,0.0,60.000000,6.0,...,43.0,12.0,Overcast,0.0,98.0,6.0,1011.0,2019-02-03 09:00:00+00:00,2257.500000,0
2,23608280,R-4d09a561,2019-01-17 07:00:00,2019-01-19 18:00:00,0,58.000,9.000,0.0,59.600000,6.0,...,43.0,12.0,Overcast,0.0,98.0,6.0,1011.0,2019-02-03 09:00:00+00:00,1831.181818,0
3,28641704,R-a55547f0,2019-01-16 07:00:00,2019-01-17 00:00:00,1,71.000,6.000,0.0,83.500000,6.0,...,43.0,12.0,Overcast,0.0,98.0,6.0,1011.0,2019-02-03 09:00:00+00:00,2276.250000,0
4,13426754,R-006548e0,2019-02-10 07:00:00,2019-02-12 00:00:00,1,51.875,4.875,0.0,72.500000,6.0,...,43.0,12.0,Overcast,0.0,98.0,6.0,1011.0,2019-02-03 09:00:00+00:00,989.459459,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,28973978,R-68283907,2019-01-10 07:00:00,2019-01-11 06:00:00,0,46.500,4.250,0.0,66.750000,6.0,...,25.0,7.0,Clear,0.0,59.0,6.0,1025.0,2019-01-11 04:00:00+00:00,1833.227273,0
10288,14969134,R-31ac88d8,2019-01-26 07:00:00,2019-01-28 00:00:00,1,71.200,5.000,0.0,70.000000,6.0,...,73.0,9.0,Patchy rain possible,0.0,72.0,6.0,1015.0,2019-01-27 18:00:00+00:00,1391.400000,0
10289,11699550,R-8ff40b97,2019-01-04 07:00:00,2019-01-04 18:00:00,1,62.000,5.000,0.0,68.333333,6.0,...,41.0,2.0,Sunny,0.0,68.0,6.0,1007.0,2019-01-04 16:00:00+00:00,2237.600000,0
10290,11191961,R-8c2f0c3a,2019-01-25 07:00:00,2019-01-26 00:00:00,0,72.000,13.500,0.0,41.500000,6.0,...,41.0,2.0,Sunny,0.0,68.0,6.0,1007.0,2019-01-04 16:00:00+00:00,2353.384615,0


In [123]:
merged_data_weather_traffic_trucks = pd.merge(merged_data_weather_traffic, trucks, on='truck_id', how='left')
merged_data_weather_traffic_trucks

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,...,humidity_y,visibility_y,pressure_y,custom_date_y,avg_no_of_vehicles,accident,truck_age,load_capacity_pounds,mileage_mpg,fuel_type
0,46155960,R-c3d9f83b,2019-02-03 07:00:00,2019-02-03 12:00:00,1,44.500,4.000,0.0,86.000000,6.0,...,98.0,6.0,1011.0,2019-02-03 09:00:00+00:00,2287.000000,1,10.0,6000.0,20.0,gas
1,23024395,R-f782724c,2019-01-10 07:00:00,2019-01-10 12:00:00,1,70.000,5.000,0.0,60.000000,6.0,...,98.0,6.0,1011.0,2019-02-03 09:00:00+00:00,2257.500000,0,11.0,10000.0,23.0,diesel
2,23608280,R-4d09a561,2019-01-17 07:00:00,2019-01-19 18:00:00,0,58.000,9.000,0.0,59.600000,6.0,...,98.0,6.0,1011.0,2019-02-03 09:00:00+00:00,1831.181818,0,10.0,6000.0,18.0,None
3,28641704,R-a55547f0,2019-01-16 07:00:00,2019-01-17 00:00:00,1,71.000,6.000,0.0,83.500000,6.0,...,98.0,6.0,1011.0,2019-02-03 09:00:00+00:00,2276.250000,0,10.0,3000.0,22.0,diesel
4,13426754,R-006548e0,2019-02-10 07:00:00,2019-02-12 00:00:00,1,51.875,4.875,0.0,72.500000,6.0,...,98.0,6.0,1011.0,2019-02-03 09:00:00+00:00,989.459459,0,9.0,4000.0,21.0,diesel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,28973978,R-68283907,2019-01-10 07:00:00,2019-01-11 06:00:00,0,46.500,4.250,0.0,66.750000,6.0,...,59.0,6.0,1025.0,2019-01-11 04:00:00+00:00,1833.227273,0,8.0,3000.0,26.0,diesel
10288,14969134,R-31ac88d8,2019-01-26 07:00:00,2019-01-28 00:00:00,1,71.200,5.000,0.0,70.000000,6.0,...,72.0,6.0,1015.0,2019-01-27 18:00:00+00:00,1391.400000,0,8.0,10000.0,26.0,diesel
10289,11699550,R-8ff40b97,2019-01-04 07:00:00,2019-01-04 18:00:00,1,62.000,5.000,0.0,68.333333,6.0,...,68.0,6.0,1007.0,2019-01-04 16:00:00+00:00,2237.600000,0,13.0,6000.0,22.0,diesel
10290,11191961,R-8c2f0c3a,2019-01-25 07:00:00,2019-01-26 00:00:00,0,72.000,13.500,0.0,41.500000,6.0,...,68.0,6.0,1007.0,2019-01-04 16:00:00+00:00,2353.384615,0,10.0,3000.0,15.0,gas


Merge merged_data with truck_data based on 'truck_id' column (Left Join)

In [124]:
final_merge = pd.merge(merged_data_weather_traffic_trucks, drivers, left_on='truck_id', right_on = 'vehicle_no', how='left')

Function to check if there is nighttime involved between arrival and departure time

In [125]:
def has_midnight(start, end):
       return int(start.date() != end.date())

Apply the function to create a new column indicating nighttime involvement

In [126]:
final_merge['is_midnight'] = final_merge.apply(lambda row: has_midnight(row['departure_date'], row['estimated_arrival']), axis=1)

In [127]:
final_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,46155960,R-c3d9f83b,2019-02-03 07:00:00,2019-02-03 12:00:00,1,44.500,4.000,0.0,86.000000,6.0,...,b92c4942-a,Brian House,male,54.0,21.0,conservative,7.0,46155960.0,41.33,0
1,23024395,R-f782724c,2019-01-10 07:00:00,2019-01-10 12:00:00,1,70.000,5.000,0.0,60.000000,6.0,...,4e89bc02-5,Albert Flores,male,49.0,8.0,proactive,3.0,23024395.0,61.12,0
2,23608280,R-4d09a561,2019-01-17 07:00:00,2019-01-19 18:00:00,0,58.000,9.000,0.0,59.600000,6.0,...,25c55dfd-7,Jose Park,male,39.0,0.0,conservative,8.0,23608280.0,40.73,1
3,28641704,R-a55547f0,2019-01-16 07:00:00,2019-01-17 00:00:00,1,71.000,6.000,0.0,83.500000,6.0,...,c61f363d-6,Shawn Johnson,male,49.0,2.0,conservative,2.0,28641704.0,47.86,1
4,13426754,R-006548e0,2019-02-10 07:00:00,2019-02-12 00:00:00,1,51.875,4.875,0.0,72.500000,6.0,...,d4763077-3,David Cook,male,49.0,14.0,conservative,6.0,13426754.0,67.24,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,28973978,R-68283907,2019-01-10 07:00:00,2019-01-11 06:00:00,0,46.500,4.250,0.0,66.750000,6.0,...,2f6b4176-3,Danny Short,male,49.0,1.0,proactive,3.0,28973978.0,57.87,1
10288,14969134,R-31ac88d8,2019-01-26 07:00:00,2019-01-28 00:00:00,1,71.200,5.000,0.0,70.000000,6.0,...,85b64b65-f,Thomas Smith,male,47.0,4.0,conservative,7.0,14969134.0,40.72,1
10289,11699550,R-8ff40b97,2019-01-04 07:00:00,2019-01-04 18:00:00,1,62.000,5.000,0.0,68.333333,6.0,...,c83501f9-b,Francis Robinson IV,male,48.0,12.0,conservative,8.0,11699550.0,48.17,0
10290,11191961,R-8c2f0c3a,2019-01-25 07:00:00,2019-01-26 00:00:00,0,72.000,13.500,0.0,41.500000,6.0,...,137b95c6-0,Stephen Thomas,male,51.0,14.0,proactive,6.0,11191961.0,59.57,1


Dropping the Duplicates

In [128]:
final_merge = final_merge.drop_duplicates()
final_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,46155960,R-c3d9f83b,2019-02-03 07:00:00,2019-02-03 12:00:00,1,44.500,4.000,0.0,86.000000,6.0,...,b92c4942-a,Brian House,male,54.0,21.0,conservative,7.0,46155960.0,41.33,0
1,23024395,R-f782724c,2019-01-10 07:00:00,2019-01-10 12:00:00,1,70.000,5.000,0.0,60.000000,6.0,...,4e89bc02-5,Albert Flores,male,49.0,8.0,proactive,3.0,23024395.0,61.12,0
2,23608280,R-4d09a561,2019-01-17 07:00:00,2019-01-19 18:00:00,0,58.000,9.000,0.0,59.600000,6.0,...,25c55dfd-7,Jose Park,male,39.0,0.0,conservative,8.0,23608280.0,40.73,1
3,28641704,R-a55547f0,2019-01-16 07:00:00,2019-01-17 00:00:00,1,71.000,6.000,0.0,83.500000,6.0,...,c61f363d-6,Shawn Johnson,male,49.0,2.0,conservative,2.0,28641704.0,47.86,1
4,13426754,R-006548e0,2019-02-10 07:00:00,2019-02-12 00:00:00,1,51.875,4.875,0.0,72.500000,6.0,...,d4763077-3,David Cook,male,49.0,14.0,conservative,6.0,13426754.0,67.24,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,28973978,R-68283907,2019-01-10 07:00:00,2019-01-11 06:00:00,0,46.500,4.250,0.0,66.750000,6.0,...,2f6b4176-3,Danny Short,male,49.0,1.0,proactive,3.0,28973978.0,57.87,1
10288,14969134,R-31ac88d8,2019-01-26 07:00:00,2019-01-28 00:00:00,1,71.200,5.000,0.0,70.000000,6.0,...,85b64b65-f,Thomas Smith,male,47.0,4.0,conservative,7.0,14969134.0,40.72,1
10289,11699550,R-8ff40b97,2019-01-04 07:00:00,2019-01-04 18:00:00,1,62.000,5.000,0.0,68.333333,6.0,...,c83501f9-b,Francis Robinson IV,male,48.0,12.0,conservative,8.0,11699550.0,48.17,0
10290,11191961,R-8c2f0c3a,2019-01-25 07:00:00,2019-01-26 00:00:00,0,72.000,13.500,0.0,41.500000,6.0,...,137b95c6-0,Stephen Thomas,male,51.0,14.0,proactive,6.0,11191961.0,59.57,1


In [129]:
final_merge = final_merge.drop(columns=['city_id_x','city_id_y','custom_date_x'])
final_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,46155960,R-c3d9f83b,2019-02-03 07:00:00,2019-02-03 12:00:00,1,44.500,4.000,0.0,86.000000,6.0,...,b92c4942-a,Brian House,male,54.0,21.0,conservative,7.0,46155960.0,41.33,0
1,23024395,R-f782724c,2019-01-10 07:00:00,2019-01-10 12:00:00,1,70.000,5.000,0.0,60.000000,6.0,...,4e89bc02-5,Albert Flores,male,49.0,8.0,proactive,3.0,23024395.0,61.12,0
2,23608280,R-4d09a561,2019-01-17 07:00:00,2019-01-19 18:00:00,0,58.000,9.000,0.0,59.600000,6.0,...,25c55dfd-7,Jose Park,male,39.0,0.0,conservative,8.0,23608280.0,40.73,1
3,28641704,R-a55547f0,2019-01-16 07:00:00,2019-01-17 00:00:00,1,71.000,6.000,0.0,83.500000,6.0,...,c61f363d-6,Shawn Johnson,male,49.0,2.0,conservative,2.0,28641704.0,47.86,1
4,13426754,R-006548e0,2019-02-10 07:00:00,2019-02-12 00:00:00,1,51.875,4.875,0.0,72.500000,6.0,...,d4763077-3,David Cook,male,49.0,14.0,conservative,6.0,13426754.0,67.24,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,28973978,R-68283907,2019-01-10 07:00:00,2019-01-11 06:00:00,0,46.500,4.250,0.0,66.750000,6.0,...,2f6b4176-3,Danny Short,male,49.0,1.0,proactive,3.0,28973978.0,57.87,1
10288,14969134,R-31ac88d8,2019-01-26 07:00:00,2019-01-28 00:00:00,1,71.200,5.000,0.0,70.000000,6.0,...,85b64b65-f,Thomas Smith,male,47.0,4.0,conservative,7.0,14969134.0,40.72,1
10289,11699550,R-8ff40b97,2019-01-04 07:00:00,2019-01-04 18:00:00,1,62.000,5.000,0.0,68.333333,6.0,...,c83501f9-b,Francis Robinson IV,male,48.0,12.0,conservative,8.0,11699550.0,48.17,0
10290,11191961,R-8c2f0c3a,2019-01-25 07:00:00,2019-01-26 00:00:00,0,72.000,13.500,0.0,41.500000,6.0,...,137b95c6-0,Stephen Thomas,male,51.0,14.0,proactive,6.0,11191961.0,59.57,1


In [130]:
final_merge=final_merge.drop(columns=['custom_date_y'])

Here x represents ORIGIN and y represnts Destination

In [131]:
final_merge.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'route_avg_temp', 'route_avg_wind_speed', 'route_avg_precip',
       'route_avg_humidity', 'route_avg_visibility', 'route_avg_pressure',
       'route_description', 'estimated_arrival_nearest_hour',
       'departure_date_nearest_hour', 'origin_id', 'destination_id',
       'distance', 'average_hours', 'temp_x', 'wind_speed_x', 'description_x',
       'precip_x', 'humidity_x', 'visibility_x', 'pressure_x', 'temp_y',
       'wind_speed_y', 'description_y', 'precip_y', 'humidity_y',
       'visibility_y', 'pressure_y', 'avg_no_of_vehicles', 'accident',
       'truck_age', 'load_capacity_pounds', 'mileage_mpg', 'fuel_type',
       'driver_id', 'name', 'gender', 'age', 'experience', 'driving_style',
       'ratings', 'vehicle_no', 'average_speed_mph', 'is_midnight'],
      dtype='object')

In [132]:
final_merge.shape

(10292, 48)